In [ ]:
!python3 -m pip install pip==24.0


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\vhgar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install numpy==1.23.5 scikit-learn==1.4.2  joblib pandas

  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
  Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)
Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl (10.6 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl (41.2 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install azureml-sdk

  Using cached backports.tempfile-1.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached msal-1.32.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached knack-0.12.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached azure_core-1.33.0-py3-none-any.whl.metadata (42 kB)
  Using cached pkginfo-1.12.1.2-py3-none-any.whl.metadata (13 kB)
  Using cached argcomplete-3.6.2-py3-none-any.whl.metadata (16 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached paramiko-3.5.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached azure_mgmt_resource-23.3.0-py3-none-any.whl.metadata (41 kB)
  Using cached azure_mgmt_containerregistry-10.3.0-py3-none-any.whl.metadata (23 kB)
  Using cached azure_mgmt_keyvault-10.3.1-py3-none-any.whl.metadata (15 kB)
  Using cached azure_

## Carga de Susctiption ID


1. Cargamos el ID de la susctripción de Azure desde las variables de entorno

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()


server_ = os.getenv('AZURE_ID')

2. Creamos un Workspace para hacer el despliegue del modelo dentro del grupo de recursos "resource_group_act3"

In [3]:
from azureml.core import Workspace

ws = Workspace.create(name="workspace_act3",
                      subscription_id = server_,
                      resource_group = "resource_group_act3",
                      create_resource_group=False,
                      location = "centralindia")


Deploying StorageAccount with name workspacstorage4964f8578.
Deploying AppInsights with name workspacinsights9d8349c9.
Deployed AppInsights with name workspacinsights9d8349c9. Took 11.92 seconds.
Deploying KeyVault with name workspackeyvault18c9f389.
Deployed StorageAccount with name workspacstorage4964f8578. Took 31.41 seconds.
Deploying Workspace with name workspace_act3.
Deployed KeyVault with name workspackeyvault18c9f389. Took 35.27 seconds.
Deployed Workspace with name workspace_act3. Took 41.4 seconds.


3. Cargamos el modelo de RandomForest y el escalador que hace onehotencode para las variables con la librería de scikit learn

In [34]:
from azureml.core.model import Model

mname = "model2"
registered_model = Model.register(model_path="../model/RandomForest_BestModel.pkl",
                                  model_name=mname,
                                  workspace=ws)

# Esto es un pipeline que incluye el modelo entrenado y el preprocesador

Registering model model2


4. Creamos un ambiente ivrtual de conda con las dependenicas necesarias para correr el modelo y el escalador.

In [35]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

virtual_env = Environment("env-actividad3")

virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['numpy==1.23.5','scikit-learn=1.4.2', 'joblib', 'pandas']
)

In [31]:
import joblib 
import pandas as pd

model = joblib.load("../model/RandomForest_BestModel.pkl")

df = pd.read_csv("../model/SalesLT.csv")

result_finals = model.predict(df)

5. Cargamos el script que genera el score del modelo, esto junto al ambiente virtual que se creó en el paso anterior.

In [36]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)


6. Desplegamos el modelo.

In [37]:
service = Model.deploy(workspace=ws,
                       name='service2',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

service.wait_for_deployment()

C:\Users\vhgar\AppData\Local\Temp\ipykernel_177004\3616185083.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,
Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: ef8b1c9d-393a-45e6-9f6d-6e4e2190093f
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. Th

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: ef8b1c9d-393a-45e6-9f6d-6e4e2190093f
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: service2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 4eec4d4b36474863b12426dc2d2fa736.azurecr.io/azureml/azureml_b10527a015c4711c3c3f4c357f6b94f6@sha256:b06d8f423ac9bbf0

In [39]:
print(service.get_logs())

None


In [8]:
from score import scorepy

print(scorepy)


import json 
import joblib
import pandas as pd
import numpy as np
import traceback
import sklearn
from azureml.core.model import Model

def embbed(d):
    try:
        required_columns = ['CustomerID', 'NameStyle', 'Title', 'FirstName', 'MiddleName',
       'LastName', 'Suffix', 'CompanyName', 'SalesPerson', 'EmailAddress',
       'Phone', 'PasswordHash', 'PasswordSalt', 'rowguid', 'ModifiedDate']
         
        missing_columns = [col for col in required_columns if col not in d.columns]
        if missing_columns:
            raise KeyError(f"Missing columns: {missing_columns}")

        return d[required_columns]

    except Exception as e:
        print(f"[ERROR] Unexpected error in embbed(): {e}")
        traceback.print_exc()
        return None

def init():
    global model, scaler

    try:
        print("[INFO] Loading model and scaler...")
        print(f"[INFO] NumPy version: {np.__version__}")
        print(f"[INFO] Scikit-learn version: {sklearn.__version__}")

        #

from utils.score import scorepy
